# El Pollo Loco locations

In [1]:
%load_ext lab_black

In [2]:
import requests
import pandas as pd
import geopandas as gpd
from vega_datasets import data
import altair as alt

/Users/stiles/.local/share/virtualenvs/notebooks-sxqKeO4C/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 10000

In [4]:
today = pd.to_datetime("today").strftime("%Y-%m-%d")

In [5]:
headers = {
    "authority": "www.elpolloloco.com",
    "accept": "application/json, text/javascript, */*; q=0.01",
    "accept-language": "en-US,en;q=0.9,es;q=0.8",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36",
    "x-requested-with": "XMLHttpRequest",
}

response = requests.get(
    "https://www.elpolloloco.com/locations/locations_json", headers=headers
)

In [6]:
src = pd.DataFrame(
    response.json(),
    columns=[
        "location_no",
        "street_address",
        "blank",
        "city",
        "state",
        "zip",
        "phone",
        "unk",
        "lat",
        "lon",
        "updated",
        "ukn",
        "slug",
        "hours",
        "blank",
        "time_zone",
        "ukn",
        "ukn",
        "closed_status",
        "ukn",
        "url",
        "id",
    ],
).sort_values("updated")

In [7]:
df = src[
    [
        "street_address",
        "city",
        "state",
        "zip",
        "phone",
        "lat",
        "lon",
        "updated",
        "slug",
        "time_zone",
    ]
].copy()

In [8]:
df["updated"] = pd.to_datetime(df["updated"]).dt.strftime("%Y-%m-%d")

In [9]:
df.head()

,street_address,city,state,zip,phone,lat,lon,updated,slug,time_zone
436,1225 S Hacienda Blvd,Hacienda Heights,CA,91745,626-330-3482,34.00844,-117.96425,1983-10-08,s-hacienda-blvd-60-fwy,America/Los_Angeles
476,2940 W Pacific Coast Hwy,Torrance,CA,90505,310-539-8792,33.79616,-118.33909,1984-08-23,s-pacific-coast-hwy-rolling-hills-pkwy,America/Los_Angeles
485,81901 State Hwy 111,Indio,CA,92201,760-342-6932,33.71240,-116.23676,1988-02-01,state-hwy-111-doctor-carrean-blvd,America/Los_Angeles
466,5055 Auburn Blvd,Sacramento,CA,95841,916-344-7939,38.66183,-121.34618,1988-05-25,auburn-blvd-madison-ave,America/Los_Angeles
479,260 S Broadway,Los Angeles,CA,90012,213-626-7975,34.05096,-118.24782,1995-05-21,s-broadway-main-st,America/Los_Angeles


In [10]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lat, df.lon))

In [11]:
states = alt.topo_feature(data.us_10m.url, feature="states")

places = (
    alt.Chart(gdf)
    .mark_circle(color="#ff9900")
    .encode(
        longitude="lon:Q",
        latitude="lat:Q",
        size=alt.value(8),
        tooltip=["street_address", "city", "zip"],
    )
    .project("albersUsa")
    .properties(width=900, height=500, title="El Pollo Loco locations")
)

states = (
    alt.Chart(states).mark_geoshape(fill="#E9E9E9", stroke="white").project("albersUsa")
)

(states + places).configure_view(strokeWidth=0)

alt.LayerChart(...)